# fetch_20newsgroupsのデータセットを使って文章分類

In [19]:
pip install nltk

     |████████████████████████████████| 1.5 MB 682 kB/s eta 0:00:01    |▊                               | 30 kB 376 kB/s eta 0:00:04     |██████▊                         | 307 kB 682 kB/s eta 0:00:02
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449904 sha256=9ce6ff625db4cb87c650bc8070993f9a036d305980349d68194ad49228c706e4
  Stored in directory: /home/jovyan/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [19]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
NewsGroup = fetch_20newsgroups()

In [22]:
data = NewsGroup.data
target = NewsGroup.target
target_names = NewsGroup.target_names

In [21]:
NewsGroup.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [31]:
data[1]

"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n"

In [30]:
NewsGroup.filenames

array(['/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/rec.autos/102994',
       '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/comp.sys.mac.hardware/51861',
       '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/comp.sys.mac.hardware/51879',
       ...,
       '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/comp.sys.ibm.pc.hardware/60695',
       '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38319',
       '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/rec.motorcycles/104440'],
      dtype='<U93')

In [24]:
target

array([7, 4, 4, ..., 3, 1, 8])

In [25]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [26]:
NewsGroup.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

### CountVectorizerを使った文章分類

In [21]:
vectorizer = CountVectorizer(min_df=1)

In [26]:
count_text = ["How to format my head disk", " Hard disk format problems "]
x = vectorizer.fit_transform(count_text)
vectorizer.get_feature_names()

['disk', 'format', 'hard', 'head', 'how', 'my', 'problems', 'to']

In [27]:
print(x.toarray().transpose())

[[1 1]
 [1 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


### fetch_20newsgroupsを使用

In [19]:
x_d, x_t, y_d, y_t = train_test_split(data, target, train_size=0.8, random_state=100) 

In [22]:
X_train = vectorizer.fit_transform(x_d)
num_sample,num_feature = X_train.shape

In [39]:
print("sample:%d, feature:%d" % (num_sample,num_feature))

sample:9051, feature:116000


In [ ]:
X_train.getrow(3).toarray()

In [76]:
class MatMul:
    def __init__(self, w):
        self.params = [w]
        self.grads = [np.zeros_like(w)]
        self.x = None
        
    def forward(self, x):
        w, = self.params
        out = np.dot(x,w)
        self.x = x
        return out
    
    def backword(self, dout):
        w, = self.params
        dx = np.dot(dout, w.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW 
        return dx

In [10]:
import numpy as np

### svdによる次元削減

In [49]:
import matplotlib.pyplot as plt

### 正の相互情報量

In [51]:
def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0
    
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            M[i,j] = max(0,pmi)
            
            if verbose:
                cnt += 1
                if cnt % (total//100) == 0:
                    print('%.1f%% done' % (100*cnt/total))
    return M

### コーパスの生成

In [8]:
def PreProcess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    courpas = np.array([word_to_id[w] for w in words])
    return courpas, word_to_id, id_to_word

### 共起行列

In [3]:
def CreateCoMatrix(courpus, VocabSize, WindowSize=1):
    CourpusSize = len(courpus)
    CoMatrix = np.zeros((VocabSize, VocabSize), dtype=np.int32)
    
    for idx, WordId in enumerate(courpus):
        for i in range(1, WindowSize+1):
            leftIdx = idx -1
            rightIdx = idx + 1
            
            if leftIdx > 0:
                left_word_id = courpus[leftIdx]
                CoMatrix[WordId, left_word_id] += 1
                
            if rightIdx < CourpusSize:
                right_word_id = courpus[rightIdx]
                CoMatrix[WordId, right_word_id] += 1
                
    return CoMatrix

### コサイン類似度

In [5]:
def cosSimilarity(x,y,eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps)
    ny = y / np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx,ny)

### 類似度の高い単語

In [40]:
def MostSimilarity(query, word_to_id, id_to_word, word_matrix, top=5):
    
    if query not in word_to_id:
        print("%s is not found." % query)
        return
    print("\n[query]" + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    vacab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cosSimilarity(word_matrix[i], query_vec)
    
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return

In [18]:
print(word_to_id)

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}


## 実際の処理

In [54]:
text = "You say goodbye and I say hello."

In [42]:
text1 = data[1]

In [55]:
courpas, word_to_id, id_to_word = PreProcess(text)
vocab_size = len(word_to_id)
C = CreateCoMatrix(courpas, vocab_size)

In [56]:
C

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]], dtype=int32)

In [46]:
MostSimilarity('you', word_to_id, id_to_word, C, top=5)


[query]you
 goodbye: 0.7071067758832467
 i: 0.7071067758832467
 hello: 0.7071067758832467
 say: 0.0
 and: 0.0


In [58]:
w = ppmi(C)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


In [60]:
U, S, V = np.linalg.svd(w)

In [61]:
print(C[0])

[0 1 0 0 0 0 0]


In [62]:
print(w[0])

[ 0. inf  0.  0.  0.  0.  0.]


In [63]:
print(U[0])

[nan  0.  0.  0.  0.  0. nan]


In [64]:
print(U[0,:2])

[nan  0.]


### Word-to-Vecを使ってみた

In [65]:
import numpy as np

In [67]:
c = np.array([1,0,0,0,0,0,0])
w = np.random.randn(7,3)
h = np.dot(c,w)
print(c)
print(w)
print(h)

[1 0 0 0 0 0 0]
[[ 0.46411331 -0.56022573 -1.31550203]
 [-0.86822163  0.46399826 -0.20555579]
 [ 1.09060313 -0.39581706  0.14605464]
 [-1.60314494  1.41141566 -1.29440288]
 [ 0.81284643 -0.74680453  0.61240945]
 [ 0.25651636 -1.42194509  0.31433607]
 [-0.34634716 -0.07689601  1.67753207]]
[ 0.46411331 -0.56022573 -1.31550203]


In [78]:
# サンプルのコンテキストデータ
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 重みの初期化
w_in = np.random.randn(7,3)
w_out = np.random.randn(3,7)

# レイヤの生成
in_layer0 = MatMul(w_in)
in_layer1 = MatMul(w_in) 
out_layer = MatMul(w_out)

In [79]:
# 順伝播
h0 = in_layer0.forward(c0) 
h1 = in_layer1.forward(c1) 
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

In [73]:
hoge = np.dot(h1,)

array([[0.5, 0. , 0.5, 0. , 0. , 0. , 0. ]])

In [74]:
h1

array([[0, 0, 1, 0, 0, 0, 0]])

In [75]:
h0

array([[1, 0, 0, 0, 0, 0, 0]])